# Convolution Neural Network

1. Train
2. Evaluate
3. Visualize Accuracy & Lost
4. Re-evaluate Model
5. Make Predictions on the Test Set
6. Visualize classification report


In [1]:
## Import Libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import pandas as pd
from PIL import Image 
from PIL.ImageDraw import Draw

In [2]:
# Define Classes
classes = ['Transformer','No-Transformer']

In [19]:
# Read in Training Data and specify image directory location in system
TRAINING_CSV_FILE = 'Data/training_data2x.csv'
TRAINING_IMAGE_DIR = 'Images/Training'

training_image_records = pd.read_csv(TRAINING_CSV_FILE)

train_image_path = TRAINING_IMAGE_DIR

# Initiate variables / arrays
train_images = []
train_targets = []
train_labels = []

# Iterrate through rows in the csv file
for index, row in training_image_records.iterrows():
    
    # Define a row to include the following headers from the csv
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    # Specify training image path (Create string) - joining the filename in csv and the path
    train_image_fullpath = os.path.join(train_image_path, filename)
    # Load the image
    train_img = keras.preprocessing.image.load_img(train_image_fullpath, target_size=(height, width))
    # Create an array of the image
    train_img_arr = keras.preprocessing.image.img_to_array(train_img)
    
    # Define xmin, ymin, xmax, and ymax
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    # Append this information in their respective arrays initiated for the 'For' loop
    train_images.append(train_img_arr)
    train_targets.append((xmin, ymin, xmax, ymax))
    train_labels.append(classes.index(class_name))

In [38]:
# Repeat process as described above but with TEST CSV and images
TEST_CSV_FILE = 'Data/test_data2.csv'
TEST_IMAGE_DIR = 'Images/Test'

test_image_records = pd.read_csv(TEST_CSV_FILE)

test_image_path = TEST_IMAGE_DIR

test_images = []
test_targets = []
test_labels = []

for index, row in test_image_records.iterrows():

    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row

    test_image_fullpath = os.path.join(test_image_path, filename)
    test_img = keras.preprocessing.image.load_img(test_image_fullpath, target_size=(height, width))
    test_img_arr = keras.preprocessing.image.img_to_array(test_img)


    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)

    test_images.append(test_img_arr)
    test_targets.append((xmin, ymin, xmax, ymax))
    test_labels.append(classes.index(class_name))

In [39]:
# Redefine appended arraya to np.arrays
train_images = np.array(train_images)
train_targets = np.array(train_targets)
train_labels = np.array(train_labels)

test_images = np.array(test_images)
test_targets = np.array(test_targets)
test_labels = np.array(test_labels)

In [40]:
# Specifying width and height of images & number of classes in this classification
width = 518
height = 388
num_classes = 2


In [41]:
# Specified convolution layers
#create the common input layer
input_shape = (height, width, 3)
input_layer = tf.keras.layers.Input(input_shape)

#create the base layers
base_layers = layers.experimental.preprocessing.Rescaling(1./255, name='bl_1')(input_layer)
base_layers = layers.Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_3')(base_layers)
base_layers = layers.Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_5')(base_layers)
base_layers = layers.Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_7')(base_layers)
base_layers = layers.Flatten(name='bl_8')(base_layers)

2021-12-07 06:15:28.406850: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
#create the classifier branch
classifier_branch = layers.Dense(128, activation='relu', name='cl_1')(base_layers)
classifier_branch = layers.Dense(num_classes, name='cl_head')(classifier_branch)  

In [43]:
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(4, activation='sigmoid', name='bb_head')(locator_branch)

In [57]:
model = tf.keras.Model(input_layer,
           outputs=[classifier_branch,locator_branch])

In [46]:
losses = {"cl_head":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
   "bb_head":tf.keras.losses.MSE}

In [47]:
model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])

In [51]:
trainTargets = {
    "cl_head": train_labels,
    "bb_head": train_targets
}
testTargets = {
    "cl_head": test_labels,
    "bb_head": test_targets
}

In [56]:
training_epochs = 5

history = model.fit(train_images, trainTargets,
            validation_data=(test_images, testTargets),
             batch_size=4,
             epochs=training_epochs,
             shuffle=True,
             verbose=1)

Epoch 1/5
75/75 [==============================] - 40s 514ms/step - loss: 1.3499 - cl_head_loss: 1.2513 - bb_head_loss: 0.0986 - cl_head_accuracy: 0.5767 - bb_head_accuracy: 0.3233 - val_loss: 0.8802 - val_cl_head_loss: 0.6934 - val_bb_head_loss: 0.1867 - val_cl_head_accuracy: 0.4000 - val_bb_head_accuracy: 0.1600
Epoch 2/5
75/75 [==============================] - 38s 502ms/step - loss: 0.7942 - cl_head_loss: 0.7073 - bb_head_loss: 0.0869 - cl_head_accuracy: 0.5900 - bb_head_accuracy: 0.3633 - val_loss: 0.7892 - val_cl_head_loss: 0.7140 - val_bb_head_loss: 0.0752 - val_cl_head_accuracy: 0.2200 - val_bb_head_accuracy: 0.1600
Epoch 3/5
75/75 [==============================] - 37s 499ms/step - loss: 0.7517 - cl_head_loss: 0.6718 - bb_head_loss: 0.0799 - cl_head_accuracy: 0.6300 - bb_head_accuracy: 0.3600 - val_loss: 0.8049 - val_cl_head_loss: 0.6985 - val_bb_head_loss: 0.1065 - val_cl_head_accuracy: 0.5100 - val_bb_head_accuracy: 0.1500
Epoch 4/5
75/75 [==============================] - 3